This lesson is optional, but very useful if you want a quick way to make a command line app with a database. My goal in creating it is to give you what I wish I had when I first started learning Python. The cmd library and sql can be combined to create a console application that can gather, process, and display data. 

In this lesson, we will show you how to construct a command line interface with sqlite and the cmd module. You can build a simple fully fledged data storage application using only the Python standard library!

Don't try to memorize this lesson. Think of it as a template that you can cut and paste.

We are going to keep our file from the previous database, cities.db. If you want, you can make a copy and use that one.

In [1]:
#The module that Python uses for the command line is called cmd, lets cut and paste some basic text here.
#Let's also import sys, because this module needs to use it to quit.
import cmd, sys
import sqlite3

In [2]:
class CitiesCMD(cmd.Cmd):
    '''
    This is the command line interface. Everything below this is for command
    line items.
    '''
       
    #all commands are prefixed with do_ or help_
    #to enter the search command at the cli you type "search"
    #to get help on it you enter "help search"
    
    intro = "CityDBShell automates the process of collecting city data."
    prompt = "(CityDBShell) "
    entry = ""
    
    
    def do_search(self, command):
        print('we need to implement the search function')
           
    def help_search(self):
        print('''Search function for finding cities.
             ''')
    
    def do_quit(self, arg):
        print("Quitting the program")
        sys.exit()
        
    def help_quit(self):
        print('Exits the program')

    def default(self, line):       
        """Called on an input line when the command prefix is not recognized.
           In that case we execute the line as Python code.
        """
        try:
            exec(line) in self._locals, self._globals
        except Exception as e:
            print(e.__class__, ":", e)  

def main():
    app = CitiesCMD().cmdloop()

Confused about what the code above does? Let me tell you a secret. I am confused about it too. So let's test it. We start it by calling the `main()` function.

In [3]:
#Let's test it out, open it up and type "search", "help search", "help", and then "quit".
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  
(CityDBShell)  
(CityDBShell)  
(CityDBShell)  exit


<class 'AttributeError'> : 'CitiesCMD' object has no attribute '_locals'


(CityDBShell)  quit


Quitting the program


SystemExit: 

/Users/thomassullivan/data_science/anaconda3/envs/cy_practice/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Notice how `cmd` uses do_search for the search command function. Then it uses help_search for the search help function

Notice how the CMD module creates a default structure for help commands!

Let's keep this simple by copying all the functions into our cmd module from the previous lesson. We will start with the database connection. We create two global variables for cursor and db, to allow the cmd module to use those variables. We connect to the database with the db. We will also add the search function to the cmd module.

In [4]:
import cmd, sys
import sqlite3

cursor = None #global variable
db = None #global variable

def find_city(id: int):
    sql = '''select name, province, country, population from cities
    where id = ?''' #
    return cur.execute(sql, (id,)).fetchall()[0]

#Notice how we add 'find_city' outside of the CMD function.

class CitiesCMD(cmd.Cmd):
    '''
    This is the command line interface. Everything below this is for command
    line items.
    '''
       
    #all commands are prefixed with do_ or help_
    #to enter the search command at the cli you type "search"
    #to get help on it you enter "help search"
    
    intro = "CityDBShell automates the process of collecting city data."
    prompt = "(CityDBShell) "
    entry = ""
    
    def do_search(self, command): #we are only implementing the search function for now.
        result = find_city(id=command)
        print(result)
           
    def help_search(self):
        print('''Search function for finding cities.
             ''')
    
    def do_quit(self, arg):
        print("Quitting the program")
        sys.exit()
        
    def help_quit(self):
        print('Exits the program')

    def default(self, line):       
        """Called on an input line when the command prefix is not recognized.
           In that case we execute the line as Python code.
        """
        try:
            exec(line) in self._locals, self._globals
        except Exception as e:
            print(e.__class__, ":", e)  

def main():
    global cur, db #writes to a global variable from within a function
    db = sqlite3.connect('cities.db') #we connected to the database!
    cur = db.cursor() #we created the cursor!
    app = CitiesCMD().cmdloop()

First, let's connect to the database automatically by adding the commands to connect to the main() function.

In [5]:
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  search 2


('Quito', 'Pichincha', 'Ecuador', 2000000)


(CityDBShell)  search 1


('Quito', 'Pichincha', 'Ecuador', 2000000)


(CityDBShell)  exit


<class 'AttributeError'> : 'CitiesCMD' object has no attribute '_locals'


(CityDBShell)  quit


Quitting the program


SystemExit: 

As you can see, we've now gotten the program to print out the information for us just by typing in a command. We've made an app. A simple one for sure but sometimes simple is best. Let's add some functionality to our app by adding the add and delete functions.

In [6]:
import cmd, sys
import sqlite3

cursor = None #global variable
db = None #global variable

def add_city(name: str, province: str, country: str, population: int) -> None: #doesn't return anything
    sql = '''insert into cities (name, province, country, population) values (?,?,?,?)'''
    cur.execute(sql, (name, province, country, population))

def find_city(id: int):
    sql = '''select name, province, country, population from cities
    where id = ?''' #
    return cur.execute(sql, (id,)).fetchall()[0]

#Notice how we add 'find_city' outside of the CMD function.

class CitiesCMD(cmd.Cmd):
    '''
    This is the command line interface. Everything below this is for command
    line items.
    '''
       
    #all commands are prefixed with do_ or help_
    #to enter the search command at the cli you type "search"
    #to get help on it you enter "help search"
    
    intro = "CityDBShell automates the process of collecting city data."
    prompt = "(CityDBShell) "
    entry = ""
    
    def do_search(self, command): #we are only implementing the search function for now.
        result = find_city(id=command)
        print(result)
           
    def help_search(self):
        print('''Search function for finding cities.
             ''')
        
    def do_add(self, command):
        command=command.split() #split the command string into multiple parts
        add_city(name=command[0], province=command[1], country=command[2], population=command[3])
    
    def do_quit(self, arg):
        print("Quitting the program")
        cur.close() #close the cursor
        db.commit() #commit the changes to the database
        db.close() #close the database
        sys.exit() #exit the program with sys.exit()
        
    def help_quit(self):
        print('Exits the program')

    def default(self, line):       
        """Called on an input line when the command prefix is not recognized.
           In that case we execute the line as Python code.
        """
        try:
            exec(line) in self._locals, self._globals
        except Exception as e:
            print(e.__class__, ":", e)  

def main():
    global cur, db #writes to a global variable from within a function
    db = sqlite3.connect('cities.db') #we connected to the database!
    cur = db.cursor() #we created the cursor!
    app = CitiesCMD().cmdloop()

In [7]:
#Let's run the main function
#then type "add Lima Lima Peru 2000000"
#then type search 4 to find Lima by its ID
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  search 1


('Quito', 'Pichincha', 'Ecuador', 2000000)


(CityDBShell)  exit


<class 'AttributeError'> : 'CitiesCMD' object has no attribute '_locals'


(CityDBShell)  quit


Quitting the program


SystemExit: 

Let us continue building our our application by adding commands to edit and delete cities from the database, as well as another function to view every city.

We will add the delete function in the code below. Notice that it is copied and pasted from the previous lesson:

`def delete_city(id: int) -> None: #type hints are optional
    sql = '''delete from cities where id = ?'''
    cur.execute(sql, (id,))`
    
And we will add a delete command to make sure it is accessible from the console:

`def do_delete(self, command):
    delete_city(id=command)`
        
The delete console command takes its input and feeds it to the `delete_city` function, with the city ID provided as user input.

When we create the delete function, we raise the possibility we might search for an article we deleted. We might also delete articles that no longer exist in the database. We protect against this by using a try...except construction. It raises an `IndexError` when we search for an item that does not exist.

In [31]:
import cmd, sys
import sqlite3

cursor = None #global variable
db = None #global variable

def add_city(name: str, province: str, country: str, population: int) -> None: #doesn't return anything
    sql = '''insert into cities (name, province, country, population) values (?,?,?,?)'''
    cur.execute(sql, (name, province, country, population))

def find_city(id: int):
    sql = '''select name, province, country, population from cities
    where id = ?''' #
    return cur.execute(sql, (id,)).fetchall()[0]

def find_all_cities(): #takes no argument because we are displaying all the cities.
    sql = '''select id, name, province, country, population from cities'''
    return cur.execute(sql).fetchall()

#Let's copy the delete_city function from chapter 2
    
def delete_city(id: int) -> None: #type hints are optional
    sql = '''delete from cities where id = ?'''
    cur.execute(sql, (id,))
    
#Notice how we add 'find_city' outside of the CMD function.

class CitiesCMD(cmd.Cmd):
    '''
    This is the command line interface. Everything below this is for command
    line items.
    '''
       
    #all commands are prefixed with do_ or help_
    #to enter the search command at the cli you type "search"
    #to get help on it you enter "help search"
    
    intro = "CityDBShell automates the process of collecting city data."
    prompt = "(CityDBShell) "
    entry = ""
    
    def do_search(self, command): #we are only implementing the search function for now.
        try:
            result = find_city(id=command)
            print(result)
        except IndexError:
            print('City does not exist')
        
    def help_search(self):
        print('''Search function for finding cities.
             ''')
        
    def do_find_all(self, command):
        result = find_all_cities()
        for item in result:
            print(item)
        
    def do_add(self, command):
        try:
            command=command.split() #split the command string into multiple parts
            add_city(name=command[0], province=command[1], country=command[2], population=command[3])    
        except IndexError:
            print('Add command entered incorrectly')
    
    def do_delete(self, command):
        try:
            delete_city(id=command)
        except IndexError:
            print('City does not exist')
    
    def do_quit(self, arg):
        print("Quitting the program")
        cur.close() #close the cursor
        db.commit() #commit the changes to the database
        db.close() #close the database
        sys.exit() #exit the program with sys.exit()
        
    def help_quit(self):
        print('Exits the program')

    def default(self, line):       
        """Called on an input line when the command prefix is not recognized.
           In that case we execute the line as Python code.
        """
        try:
            exec(line) in self._locals, self._globals
        except Exception as e:
            print(e.__class__, ":", e)  

def main():
    global cur, db #writes to a global variable from within a function
    db = sqlite3.connect('cities.db') #we connected to the database!
    cur = db.cursor() #we created the cursor!
    app = CitiesCMD().cmdloop()

In [25]:
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  add villcabamba loja ecuador d
(CityDBShell)  search 4


('villcabamba', 'loja', 'ecuador', 'd')


(CityDBShell)  quit


Quitting the program


SystemExit: 

So typing in `find_all` displays all of the information about the cities in a Tuple. For a short database like this, it is enough to be able to search for and edit entries specifically. What happens if we ask for help with `find_all` or `add_city`.

Now let's create an update function.

In [16]:
#Try deleting a city and then searching for it, you should get a printed message that the city does not exist

Now that we have the add function completed, we need to do two more things to finish our cmd/sqlite command line interface.

1) Add an update function.
2) Document the commands.

To illustrate what I mean by documenting commands, let's go back to our app and display the help menu.

In [18]:
main() #calls the most recent version of the app in a Jupyter notebook. 

CityDBShell automates the process of collecting city data.


(CityDBShell)  search 4


City does not exist


(CityDBShell)  exit 4


<class 'SyntaxError'> : invalid syntax (<string>, line 1)


(CityDBShell)  quit 4


Quitting the program


SystemExit: 

Notice how the `help` command displays certain commands as "undocumented commands." These are because there is no help function associated with them. Let's add the help functions to this program.

In [35]:
import cmd, sys
import sqlite3

cursor = None #global variable
db = None #global variable

def add_city(name: str, province: str, country: str, population: int) -> None: #doesn't return anything
    sql = '''insert into cities (name, province, country, population) values (?,?,?,?)'''
    cur.execute(sql, (name, province, country, population))

def find_city(id: int):
    sql = '''select name, province, country, population from cities
    where id = ?''' #
    return cur.execute(sql, (id,)).fetchall()[0]

def find_all_cities(): #takes no argument because we are displaying all the cities.
    sql = '''select id, name, province, country, population from cities'''
    return cur.execute(sql).fetchall()

#Let's copy the delete_city function from chapter 2
    
def delete_city(id: int) -> None: #type hints are optional
    sql = '''delete from cities where id = ?'''
    cur.execute(sql, (id,))
    
#Notice how we add 'find_city' outside of the CMD function.

class CitiesCMD(cmd.Cmd):
    '''
    This is the command line interface. Everything below this is for command
    line items.
    '''
       
    #all commands are prefixed with do_ or help_
    #to enter the search command at the cli you type "search"
    #to get help on it you enter "help search"
    
    intro = "CityDBShell automates the process of collecting city data."
    prompt = "(CityDBShell) "
    entry = ""
    
    def do_search(self, command): #we are only implementing the search function for now.
        try:
            result = find_city(id=command)
            print(result)
        except IndexError:
            print('City does not exist')
        
    def help_search(self):
        print('''Search function for finding cities.
             ''')
        
    def do_find_all(self, command):
        result = find_all_cities()
        for item in result:
            print(item)
        
    def do_add(self, command):
        try:
            command=command.split() #split the command string into multiple parts
            add_city(name=command[0], province=command[1], country=command[2], population=command[3])
        except IndexError:
            print('Add command entered incorrectly')
        
    def help_add(self): #we don't include any arguments in the help_add function.
        print('Adds a new city to the database')
        print('Example: add [name] [province] [country] [population]')
        print('Name, province, and country are strings, population is an integer')
        print('An exception will NOT be raised if the user enters a string for population.')
        
    def update_name(self, command):
        command=command.split()
        
    def do_delete(self, command):
        try:
            delete_city(id=command)
        except IndexError:
            print('City does not exist')
    
    def do_quit(self, arg):
        print("Quitting the program")
        cur.close() #close the cursor
        db.commit() #commit the changes to the database
        db.close() #close the database
        sys.exit() #exit the program with sys.exit()
        
    def help_quit(self):
        print('Exits the program')

    def default(self, line):       
        """Called on an input line when the command prefix is not recognized.
           In that case we execute the line as Python code.
        """
        try:
            exec(line) in self._locals, self._globals
        except Exception as e:
            print(e.__class__, ":", e)  

def main():
    global cur, db #writes to a global variable from within a function
    db = sqlite3.connect('cities.db') #we connected to the database!
    cur = db.cursor() #we created the cursor!
    app = CitiesCMD().cmdloop()

In [34]:
#Now let's run it again and see how the help function works now.
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  exit


<class 'AttributeError'> : 'CitiesCMD' object has no attribute '_locals'


(CityDBShell)  quit


Quitting the program


SystemExit: 

Now that we have added the help functions, we have only one more thing to do. The update function. To make it simplier, we will have four commands, one for each of the attributes of the city. `update_name` updates the name, while the other commands update their respective fields. Let's also go ahead and add help for these functions.

In [ ]:
import cmd, sys
import sqlite3

cursor = None #global variable
db = None #global variable

def add_city(name: str, province: str, country: str, population: int) -> None: #doesn't return anything
    sql = '''insert into cities (name, province, country, population) values (?,?,?,?)'''
    cur.execute(sql, (name, province, country, population))

def find_city(id: int):
    sql = '''select name, province, country, population from cities
    where id = ?''' #
    return cur.execute(sql, (id,)).fetchall()[0]

def find_all_cities(): #takes no argument because we are displaying all the cities.
    sql = '''select id, name, province, country, population from cities'''
    return cur.execute(sql).fetchall()

#Notice how we have copied the update_city function from chapter 2, but pasted it outside the cmd class.

def update_city(id, name=None, province=None, country=None, population=None):
    sql = '''
    update cities set name =?, province=?, country=?, population=? where id =?'''
    data = find_city(id) #we are calling "find_city" to get info on the city we want to update.
    if not name: name=data[0]
    if not province: province = data[1]
    if not country: country = data[2]
    if not population: population = data[3]
    cur.execute(sql, (name, province, country, population, id))

#Let's copy the delete_city function from chapter 2
    
def delete_city(id: int) -> None: #type hints are optional
    sql = '''delete from cities where id = ?'''
    cur.execute(sql, (id,))
    
#Notice how we add 'find_city' outside of the CMD function.

class CitiesCMD(cmd.Cmd):
    '''
    This is the command line interface. Everything below this is for command
    line items.
    '''
       
    #all commands are prefixed with do_ or help_
    #to enter the search command at the cli you type "search"
    #to get help on it you enter "help search"
    
    intro = "CityDBShell automates the process of collecting city data."
    prompt = "(CityDBShell) "
    entry = ""
    
    def do_search(self, command): #we are only implementing the search function for now.
        try:
            result = find_city(id=command)
            print(result)
        except IndexError:
            print('City does not exist')
        
    def help_search(self):
        print('''Search function for finding cities.
             ''')
        
    def do_find_all(self, command):
        result = find_all_cities()
        for item in result:
            print(item)
        
    def do_add(self, command):
        try:
            command=command.split() #split the command string into multiple parts
            add_city(name=command[0], province=command[1], country=command[2], population=command[3])
        except IndexError:
            print('Add command entered incorrectly')
        
    def help_add(self): #we don't include any arguments in the help_add function.
        print('Adds a new city to the database')
        print('Example: add [name] [province] [country] [population]')
        print('Name, province, and country are strings, population is an integer')
        print('An exception will NOT be raised if the user enters a string for population.')
        
    def do_update_name(self, command):
        command=command.split()
        print(command)
        update_city(id=command[0], name=command[1])
        
    def help_update_name(self):
        print('updates the name of an entry')
        print('Example:')
        print('update_name [id] [new_name]')
        print('update_name 4 Villcabamba')
        
    def do_update_province(self, command):
        command=command.split()
        print(command)
        update_city(id=command[0], province=command[1])
        
    def help_update_province(self):
        print('updates the province of an entry')
        print('Example:')
        print('update_name [id] [new_name]')
        print('update_name 4 Loja')
        
    def do_update_country(self, command):
        command=command.split()
        print(command)
        update_city(id=command[0], country=command[1])
        
    def help_update_country(self, command):
        print('updates the country of an entry')
        print('Example:')
        print('update_name [id] [new_name]')
        print('update_name 4 Villcabamba')
        
    def do_update_population(self, command):
        command=command.split()
        print(command)
        update_city(id=command[0], population=command[1])
        
    def help_update_population(self, command):
        print('updates the population of an entry')
        print('Example:')
        print('update_population [id] [new_name]')
        print('update_population 4 4000')
        
    def do_delete(self, command):
        try:
            delete_city(id=command)
        except IndexError:
            print('City does not exist')
    
    def do_quit(self, arg):
        print("Quitting the program")
        cur.close() #close the cursor
        db.commit() #commit the changes to the database
        db.close() #close the database
        sys.exit() #exit the program with sys.exit()
        
    def help_quit(self):
        print('Exits the program')

    def default(self, line):       
        """Called on an input line when the command prefix is not recognized.
           In that case we execute the line as Python code.
        """
        try:
            exec(line) in self._locals, self._globals
        except Exception as e:
            print(e.__class__, ":", e)  

def main():
    global cur, db #writes to a global variable from within a function
    db = sqlite3.connect('cities.db') #we connected to the database!
    cur = db.cursor() #we created the cursor!
    app = CitiesCMD().cmdloop()

In [4]:
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  help



Documented commands (type help <topic>):
add   quit    update_country  update_population
help  search  update_name     update_province  

Undocumented commands:
delete  find_all



(CityDBShell)  exit


<class 'AttributeError'> : 'CitiesCMD' object has no attribute '_locals'


(CityDBShell)  quit


Quitting the program


SystemExit: 

Now that we have the basic functions created, let's try to make an update function that works for any of the types of updates. This function is harder to write than it is to use.

In [8]:
import cmd, sys
import sqlite3

cursor = None #global variable
db = None #global variable

def add_city(name: str, province: str, country: str, population: int) -> None: #doesn't return anything
    sql = '''insert into cities (name, province, country, population) values (?,?,?,?)'''
    cur.execute(sql, (name, province, country, population))

def find_city(id: int):
    sql = '''select name, province, country, population from cities
    where id = ?''' #
    return cur.execute(sql, (id,)).fetchall()[0]

def find_all_cities(): #takes no argument because we are displaying all the cities.
    sql = '''select id, name, province, country, population from cities'''
    return cur.execute(sql).fetchall()

#Notice how we have copied the update_city function from chapter 2, but pasted it outside the cmd class.

def update_city(id, name=None, province=None, country=None, population=None):
    sql = '''
    update cities set name =?, province=?, country=?, population=? where id =?'''
    data = find_city(id) #we are calling "find_city" to get info on the city we want to update.
    if not name: name=data[0]
    if not province: province = data[1]
    if not country: country = data[2]
    if not population: population = data[3]
    cur.execute(sql, (name, province, country, population, id))

#Let's copy the delete_city function from chapter 2
    
def delete_city(id: int) -> None: #type hints are optional
    sql = '''delete from cities where id = ?'''
    cur.execute(sql, (id,))
    
#Notice how we add 'find_city' outside of the CMD function.

class CitiesCMD(cmd.Cmd):
    '''
    This is the command line interface. Everything below this is for command
    line items.
    '''
       
    #all commands are prefixed with do_ or help_
    #to enter the search command at the cli you type "search"
    #to get help on it you enter "help search"
    
    intro = "CityDBShell automates the process of collecting city data."
    prompt = "(CityDBShell) "
    entry = ""
    
    def do_search(self, command): #we are only implementing the search function for now.
        try:
            result = find_city(id=command)
            print(result)
        except IndexError:
            print('City does not exist')
        
    def help_search(self):
        print('''Search function for finding cities.
             ''')
        
    def do_find_all(self, command):
        result = find_all_cities()
        for item in result:
            print(item)
        
    def do_add(self, command):
        try:
            command=command.split() #split the command string into multiple parts
            add_city(name=command[0], province=command[1], country=command[2], population=command[3])
        except IndexError:
            print('Add command entered incorrectly')
        
    def help_add(self): #we don't include any arguments in the help_add function.
        print('Adds a new city to the database')
        print('Example: add [name] [province] [country] [population]')
        print('Name, province, and country are strings, population is an integer')
        print('An exception will NOT be raised if the user enters a string for population.')
        
    def do_update(self, command):
        command = command.split()
        print(command) #we want to see the command as its being entered for testing purposes
        #update_id = command[0]
        update_type = command[1] #The second thing you enter is the type of update e.g. update 2 population 2000000 creates the variable
                                #'command' as a list of three elements. The first element is the id of the city to update, the second is the 
                                #type of update e.g. description, while the third is the new value
        #update_value = command[2]
        if update_type == 'name':
            update_city(id=command[0], name=command[2]) #we take the third element as the new value
            print('city updated')
        elif update_type == 'province':
            update_city(id=command[0], province=command[2])
        elif update_type == 'country':
            update_city(id=command[0], country=command[2])
        elif update_type == 'population':
            update_city(id=command[0], population=command[2])
        
#    def do_update_name(self, command):
#        command=command.split()
#        print(command)
#        update_city(id=command[0], name=command[1])
        
#    def help_update_name(self):
#        print('updates the name of an entry')
#        print('Example:')
#        print('update_name [id] [new_name]')
#        print('update_name 4 Villcabamba')
        
#    def do_update_province(self, command):
#        command=command.split()
#        print(command)
#        update_city(id=command[0], province=command[1])
        
#    def help_update_province(self):
#        print('updates the province of an entry')
#        print('Example:')
#        print('update_name [id] [new_name]')
#        print('update_name 4 Loja')
        
#    def do_update_country(self, command):
#        command=command.split()
#        print(command)
#        update_city(id=command[0], country=command[1])
        
#    def help_update_country(self, command):
#        print('updates the country of an entry')
#        print('Example:')
#        print('update_name [id] [new_name]')
#        print('update_name 4 Villcabamba')
#        
#    def do_update_population(self, command):
#        command=command.split()
#        print(command)
#        update_city(id=command[0], population=command[1])
        
#    def help_update_population(self, command):
#        print('updates the population of an entry')
#        print('Example:')
#        print('update_population [id] [new_name]')
#        print('update_population 4 4000')
        
    def do_delete(self, command):
        try:
            delete_city(id=command)
        except IndexError:
            print('City does not exist')
    
    def do_quit(self, arg):
        print("Quitting the program")
        cur.close() #close the cursor
        db.commit() #commit the changes to the database
        db.close() #close the database
        sys.exit() #exit the program with sys.exit()
        
    def help_quit(self):
        print('Exits the program')

    def default(self, line):       
        """Called on an input line when the command prefix is not recognized.
           In that case we execute the line as Python code.
        """
        try:
            exec(line) in self._locals, self._globals
        except Exception as e:
            print(e.__class__, ":", e)  

def main():
    global cur, db #writes to a global variable from within a function
    db = sqlite3.connect('cities.db') #we connected to the database!
    cur = db.cursor() #we created the cursor!
    app = CitiesCMD().cmdloop()

In [7]:
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  help



Documented commands (type help <topic>):
add   quit    update_country  update_population
help  search  update_name     update_province  

Undocumented commands:
delete  find_all  update



(CityDBShell)  update 1 province Loja


['1', 'province', 'Loja']


...

(CityDBShell)  search 1


('Villcabamba', 'Loja', 'Ecuador', 4000)


(CityDBShell)  delete 4
(CityDBShell)  quit


Quitting the program


SystemExit: 

If we have written the update function correctly. We can now update the data directly from the command line. This means we can now comment out our existing update functions. The new update function is harder to write than the previous functions, but easier to use.

Let us rewrite our application one last time. This time, let's add some safety features to prevent the user from:

1) Accidentally entering a string for population

We can do this by converting the string input to an integer, which will raise a ValueError if we try to set it as an alphanumeric character.
We can then catch the ValueError exception and inform the user with a prompt that they must enter an integer for population.

2) Deleting a city when they don't really mean to do so.

In [1]:
import cmd, sys
import sqlite3

cursor = None #global variable
db = None #global variable

def add_city(name: str, province: str, country: str, population: int) -> None: #doesn't return anything
    sql = '''insert into cities (name, province, country, population) values (?,?,?,?)'''
    cur.execute(sql, (name, province, country, population))

def find_city(id: int):
    sql = '''select name, province, country, population from cities
    where id = ?''' #
    return cur.execute(sql, (id,)).fetchall()[0]

def find_all_cities(): #takes no argument because we are displaying all the cities.
    sql = '''select id, name, province, country, population from cities'''
    return cur.execute(sql).fetchall()

#Notice how we have copied the update_city function from chapter 2, but pasted it outside the cmd class.

def update_city(id, name=None, province=None, country=None, population=None):
    sql = '''
    update cities set name =?, province=?, country=?, population=? where id =?'''
    data = find_city(id) #we are calling "find_city" to get info on the city we want to update.
    if not name: name=data[0]
    if not province: province = data[1]
    if not country: country = data[2]
    if not population: population = data[3]
    cur.execute(sql, (name, province, country, population, id))

#Let's copy the delete_city function from chapter 2
    
def delete_city(id: int) -> None: #type hints are optional
    sql = '''delete from cities where id = ?'''
    cur.execute(sql, (id,))
    
#Notice how we add 'find_city' outside of the CMD function.

class CitiesCMD(cmd.Cmd):
    '''
    This is the command line interface. Everything below this is for command
    line items.
    '''
       
    #all commands are prefixed with do_ or help_
    #to enter the search command at the cli you type "search"
    #to get help on it you enter "help search"
    
    intro = "CityDBShell automates the process of collecting city data."
    prompt = "(CityDBShell) "
    entry = ""
    
    def do_search(self, command): #we are only implementing the search function for now.
        try:
            result = find_city(id=command)
            print(result)
        except IndexError:
            print('City does not exist')
        
    def help_search(self):
        print('''Search function for finding cities.
             ''')
        
    def do_find_all(self, command):
        result = find_all_cities()
        for item in result:
            print(item)
        
    def do_add(self, command):
        try:
            command=command.split() #split the command string into multiple parts
            add_city(name=command[0], province=command[1], country=command[2], population=command[3])
        except IndexError:
            print('Add command entered incorrectly')
        
    def help_add(self): #we don't include any arguments in the help_add function.
        print('Adds a new city to the database')
        print('Example: add [name] [province] [country] [population]')
        print('Name, province, and country are strings, population is an integer')
        print('An exception will NOT be raised if the user enters a string for population.')
        
    def do_update(self, command):
        try:
            command = command.split()
            print(command) #we want to see the command as its being entered for testing purposes
            update_id = command[0]
            update_type = command[1]
            update_value = command[2]
            if update_type == 'name':
                update_city(id=command[0], name=command[2]) #we take the third element as the new value
                print('city updated')
            elif update_type == 'province':
                update_city(id=update_id, province=update_value)
            elif update_type == 'country':
                update_city(id=update_id, country=update_value)
            elif update_type == 'population':
                try:
                    update_value = int(update_value) #we convert it into an integer
                    update_city(id=command[0], population=update_value)
                except ValueError:
                    print('Population must be a numeric value')
        except IndexError:
            print('Update command entered incorrectly')
        
    def do_delete(self, command):
        try:
            delete_city(id=command)
        except IndexError:
            print('City does not exist')
    
    def do_quit(self, arg):
        print("Quitting the program")
        cur.close() #close the cursor
        db.commit() #commit the changes to the database
        db.close() #close the database
        sys.exit() #exit the program with sys.exit()
        
    def help_quit(self):
        print('Exits the program')

    def default(self, line):       
        """Called on an input line when the command prefix is not recognized.
           In that case we execute the line as Python code.
        """
        try:
            exec(line) in self._locals, self._globals
        except Exception as e:
            print(e.__class__, ":", e)  

def main():
    global cur, db #writes to a global variable from within a function
    db = sqlite3.connect('cities.db') #we connected to the database!
    cur = db.cursor() #we created the cursor!
    app = CitiesCMD().cmdloop()

In [16]:
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  update 2 population f


['2', 'population', 'f']
Population must be a numeric value


(CityDBShell)  exit


<class 'AttributeError'> : 'CitiesCMD' object has no attribute '_locals'


(CityDBShell)  quit


Quitting the program


SystemExit: 

Let's do a few more final steps to get this basic application working.

1) Complete the help for all functions.
2) Manage any potential difficulties with incorrect data entry.
3) Make sure the user really wants to delete something.

In [7]:
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  add washington dc usa 700000
(CityDBShell)  exit


<class 'AttributeError'> : 'CitiesCMD' object has no attribute '_locals'


(CityDBShell)  quit


Quitting the program


SystemExit: 

/Users/thomassullivan/data_science/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
import cmd, sys
import sqlite3

cursor = None #global variable
db = None #global variable

def add_city(name: str, province: str, country: str, population: int) -> None: #doesn't return anything
    sql = '''insert into cities (name, province, country, population) values (?,?,?,?)'''
    cur.execute(sql, (name, province, country, population))

def find_city(id: int):
    #with closing(cur()) as c:
    sql = '''select name, province, country, population from cities
    where id = ?''' #
    return cur.execute(sql, (id,)).fetchall()[0]

def find_all_cities(): #takes no argument because we are displaying all the cities.
    sql = '''select id, name, province, country, population from cities'''
    return cur.execute(sql).fetchall()

#Notice how we have copied the update_city function from chapter 2, but pasted it outside the cmd class.

def update_city(id, name=None, province=None, country=None, population=None):
    sql = '''
    update cities set name =?, province=?, country=?, population=? where id =?'''
    data = find_city(id) #we are calling "find_city" to get info on the city we want to update.
    if not name: name=data[0]
    if not province: province = data[1]
    if not country: country = data[2]
    if not population: population = data[3]
    cur.execute(sql, (name, province, country, population, id))

#Let's copy the delete_city function from chapter 2
    
def delete_city(id: int) -> None: #type hints are optional
    sql = '''delete from cities where id = ?'''
    cur.execute(sql, (id,))
    
#Notice how we add 'find_city' outside of the CMD function.

class CitiesCMD(cmd.Cmd):
    '''
    This is the command line interface. Everything below this is for command
    line items.
    '''
       
    #all commands are prefixed with do_ or help_
    #to enter the search command at the cli you type "search"
    #to get help on it you enter "help search"
    
    intro = "CityDBShell automates the process of collecting city data."
    prompt = "(CityDBShell) "
    entry = ""
    
    def do_search(self, command): #we are only implementing the search function for now.
        try:
            result = find_city(id=command)
            print(result)
        except IndexError:
            print('City does not exist')
        
    def help_search(self):
        print('''Search function for finding cities.
             ''')
        
    def do_find_all(self, command):
        result = find_all_cities()
        for item in result:
            print(item)
            
    def help_find_all(self):
        print('Displays all the cities in the database')
        print('Enter "find_all" without any commands')
        
    def do_add(self, command):
        try:
            command=command.split() #split the command string into multiple parts
            add_city(name=command[0], province=command[1], country=command[2], population=command[3])
        except IndexError:
            print('Add command entered incorrectly')
        
    def help_add(self): #we don't include any arguments in the help_add function.
        print('Adds a new city to the database')
        print('Example: add [name] [province] [country] [population]')
        print('Name, province, and country are strings, population is an integer')
        print('An exception will NOT be raised if the user enters a string for population.')
        
    def do_update(self, command):
        try:
            command = command.split()
            print(command) #we want to see the command as its being entered for testing purposes
            update_id = command[0]
            update_type = command[1]
            update_value = command[2]
            if update_type == 'name':
                update_city(id=command[0], name=command[2]) #we take the third element as the new value
                print('city updated')
            elif update_type == 'province':
                update_city(id=update_id, province=update_value)
            elif update_type == 'country':
                update_city(id=update_id, country=update_value)
            elif update_type == 'population':
                try:
                    update_value = int(update_value) #we convert it into an integer
                    if update_value < 0: #we make sure it cannot be less than zero
                        raise ValueError
                    else:
                        update_city(id=update_id, population=update_value)
                except ValueError:
                    print('Population must be a numeric value greater than zero')
        except IndexError:
            print('Update command entered incorrectly')
    
    def help_update(self):
        print('''
        Updates an entry in the database
        example:
        update [id] [update_type] [new_value]
        "update 3 province Loha" changes the name of the province for entry 3 to Loha
        ''')
        
    def do_delete(self, command): #Let's make sure the user really wants to delete something
        try:
            result = find_city(id=command)
            input_decision = input('Are you sure you want to delete "{0}" from the database? (y/n)'.format(result[0]))
            if input_decision == 'y':
                delete_city(id=command)
            else:
                print('Delete cancelled, returning to main menu')
        except IndexError:
            print('City does not exist')
            
    def help_delete(self, command):
        print('deletes a city from the database')
        print('example:')
        print('"delete 4" will delete the 4th city in the database')
    
    def do_quit(self, arg):
        print("Quitting the program")
        cur.close() #close the cursor
        db.commit() #commit the changes to the database
        db.close() #close the database
        sys.exit() #exit the program with sys.exit()
        
    def help_quit(self):
        print('Exits the program')

    def default(self, line):       
        """Called on an input line when the command prefix is not recognized.
           In that case we execute the line as Python code.
        """
        try:
            exec(line) in self._locals, self._globals
        except Exception as e:
            print(e.__class__, ":", e)  

def main():
    global cur, db #writes to a global variable from within a function
    db = sqlite3.connect('cities.db') #we connected to the database!
    cur = db.cursor() #we created the cursor!
    app = CitiesCMD().cmdloop()

In [6]:
main()

CityDBShell automates the process of collecting city data.


(CityDBShell)  search 


City does not exist


(CityDBShell)  quit


Quitting the program


SystemExit: 